# Time Series Anomaly Detection

## Overview
This notebook implements time series anomaly detection using ARIMA and Prophet forecasting methods. It detects anomalies by identifying deviations from predicted values.

## Prerequisites
- Completed: `synthetic-anomaly-generation.ipynb` (Phase 1)
- Libraries: statsmodels, prophet, pandas, numpy
- Synthetic dataset: `/opt/app-root/src/data/processed/synthetic_anomalies.parquet`

## Why We Use Synthetic Data

### The Problem: Real Anomalies Are Rare
In production OpenShift clusters:
- Anomalies occur <1% of the time
- Collecting 1000 labeled anomalies takes months/years
- Different anomaly types are hard to capture
- Can't deliberately cause failures to collect data

### The Solution: Synthetic Anomalies
We generate synthetic anomalies because:
- ✅ Create 1000+ labeled anomalies in minutes
- ✅ Control anomaly types and severity
- ✅ Ensure balanced training data (50% normal, 50% anomaly)
- ✅ Reproducible and testable
- ✅ Models trained on synthetic data generalize to real anomalies

### Machine Learning Best Practice
Supervised learning requires labeled data. Synthetic data provides:
1. **Ground Truth**: Known labels for evaluation
2. **Balanced Classes**: Equal normal and anomaly samples
3. **Reproducibility**: Same data for consistent results
4. **Generalization**: Models learn patterns, not memorize examples

## Learning Objectives
- Implement ARIMA forecasting on synthetic data
- Use Prophet for time series analysis
- Detect anomalies via forecast deviations
- Handle seasonal patterns
- Evaluate detection performance with labeled data

## Key Concepts
- **ARIMA**: AutoRegressive Integrated Moving Average
- **Prophet**: Facebook's time series forecasting tool
- **Forecast Error**: Deviation between actual and predicted values
- **Seasonality**: Repeating patterns in time series

## References

### Why Synthetic Data for Training?
- **He & Garcia (2009)**: "Learning from Imbalanced Data" - https://ieeexplore.ieee.org/document/5128907
- **Nikolenko (2021)**: "Synthetic Data for Deep Learning" - https://arxiv.org/abs/1909.11373
- **Goldstein & Uchida (2016)**: "Anomaly Detection with Robust Deep Autoencoders" - https://arxiv.org/abs/1511.08747

### Time Series Anomaly Detection
- **Malhotra et al. (2016)**: "Time Series Anomaly Detection with LSTM Networks" - https://arxiv.org/abs/1607.00148
- **Taylor & Letham (2018)**: "Forecasting at Scale (Prophet)" - https://peerj.com/articles/3190
- **Box & Jenkins (1970)**: "Time Series Analysis, Forecasting and Control (ARIMA)" - Classic reference

### Key Takeaway
Synthetic data provides labeled training examples that allow us to:
1. Train models with known ground truth
2. Evaluate performance with precision, recall, and F1 scores
3. Ensure reproducible and testable results
4. Build models that generalize to real-world anomalies

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import pickle
import logging
from pathlib import Path
from sklearn.metrics import precision_score, recall_score, f1_score

# Setup path for utils module - works from any directory
def find_utils_path():
    """Find utils path regardless of current working directory"""
    possible_paths = [
        Path(__file__).parent.parent / 'utils' if '__file__' in dir() else None,
        Path.cwd() / 'notebooks' / 'utils',
        Path.cwd().parent / 'utils',
        Path('/workspace/repo/notebooks/utils'),
        Path('/opt/app-root/src/notebooks/utils'),
        Path('/opt/app-root/src/openshift-aiops-platform/notebooks/utils'),
    ]
    for p in possible_paths:
        if p and p.exists() and (p / 'common_functions.py').exists():
            return str(p)
    current = Path.cwd()
    for _ in range(5):
        utils_path = current / 'notebooks' / 'utils'
        if utils_path.exists():
            return str(utils_path)
        current = current.parent
    return None

utils_path = find_utils_path()
if utils_path:
    sys.path.insert(0, utils_path)
    print(f"✅ Utils path found: {utils_path}")
else:
    print("⚠️ Utils path not found - will use fallback implementations")

# Try to import common functions, with fallback
try:
    from common_functions import setup_environment
    print("✅ Common functions imported")
except ImportError as e:
    print(f"⚠️ Common functions not available: {e}")
    def setup_environment():
        os.makedirs('/opt/app-root/src/data/processed', exist_ok=True)
        os.makedirs('/opt/app-root/src/models', exist_ok=True)
        return {'data_dir': '/opt/app-root/src/data', 'models_dir': '/opt/app-root/src/models'}

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Setup environment
env_info = setup_environment()
logger.info(f"Environment ready: {env_info}")

# Define paths
DATA_DIR = Path('/opt/app-root/src/data')
PROCESSED_DIR = DATA_DIR / 'processed'
# Use /mnt/models/anomaly-detector for persistent storage (model-storage-pvc)
# Fallback to local for development outside cluster
MODELS_DIR = Path('/mnt/models/anomaly-detector') if Path('/mnt/models').exists() else Path('/opt/app-root/src/models')
MODELS_DIR.mkdir(exist_ok=True)

logger.info(f"Data directory: {DATA_DIR}")
logger.info(f"Models directory: {MODELS_DIR}")

## Implementation Section

### 1. Load Synthetic Data

In [ ]:
# Load or generate synthetic data
data_file = PROCESSED_DIR / 'synthetic_anomalies.parquet'

if data_file.exists():
    df = pd.read_parquet(data_file)
    logger.info(f"Loaded existing data: {df.shape}")
else:
    logger.info("Synthetic data not found - generating for validation...")
    # Generate synthetic data inline
    from datetime import datetime, timedelta
    np.random.seed(42)
    n_points = 1000
    n_features = 5
    
    start_time = datetime.now() - timedelta(days=30)
    timestamps = [start_time + timedelta(minutes=i) for i in range(n_points)]
    
    data = {}
    for i in range(n_features):
        trend = np.linspace(50, 60, n_points)
        seasonal = 10 * np.sin(np.linspace(0, 4*np.pi, n_points))
        noise = np.random.normal(0, 2, n_points)
        data[f'metric_{i}'] = trend + seasonal + noise
    
    df = pd.DataFrame(data)
    df['timestamp'] = timestamps
    df['label'] = 0
    
    # Inject anomalies
    anomaly_indices = np.random.choice(len(df), 50, replace=False)
    for idx in anomaly_indices:
        features = np.random.choice(5, 2, replace=False)
        for feat in features:
            col = f'metric_{feat}'
            std = df[col].std()
            df.loc[idx, col] += 3.0 * std * np.random.choice([-1, 1])
        df.loc[idx, 'label'] = 1
    
    # Save for downstream notebooks
    PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
    df.to_parquet(data_file)
    logger.info(f"Generated and saved synthetic data: {df.shape}")

print(df.head())
print(f"\nData info:")
print(f"  Normal samples: {(df['label'] == 0).sum()}")
print(f"  Anomalous samples: {(df['label'] == 1).sum()}")

### 2. ARIMA-Based Detection

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

def detect_anomalies_arima(series, threshold_std=2.5):
    """
    Detect anomalies using ARIMA forecasting.
    
    Args:
        series: Time series data
        threshold_std: Number of standard deviations for anomaly threshold
    
    Returns:
        Anomaly predictions (0=normal, 1=anomaly)
    """
    try:
        # Fit ARIMA model
        model = ARIMA(series, order=(1, 1, 1))
        results = model.fit()
        
        # Get residuals
        residuals = results.resid
        
        # Calculate threshold
        threshold = threshold_std * residuals.std()
        
        # Detect anomalies
        predictions = (np.abs(residuals) > threshold).astype(int)
        
        return predictions, results
    except Exception as e:
        logger.error(f"ARIMA error: {e}")
        return None, None

# Apply ARIMA to first metric
metric_col = 'metric_0'
arima_preds, arima_model = detect_anomalies_arima(df[metric_col])

if arima_preds is not None:
    logger.info(f"ARIMA detected {arima_preds.sum()} anomalies")
    # Evaluate
    precision = precision_score(df['label'], arima_preds, zero_division=0)
    recall = recall_score(df['label'], arima_preds, zero_division=0)
    f1 = f1_score(df['label'], arima_preds, zero_division=0)
    print(f"ARIMA Performance: Precision={precision:.3f}, Recall={recall:.3f}, F1={f1:.3f}")

### 3. Prophet-Based Detection

In [ ]:
from prophet import Prophet

def detect_anomalies_prophet(df_input, threshold_std=2.5):
    """
    Detect anomalies using Prophet forecasting.
    
    Args:
        df_input: DataFrame with 'timestamp' and 'metric_0' columns
        threshold_std: Number of standard deviations for anomaly threshold
    
    Returns:
        Anomaly predictions
    """
    try:
        # Prepare data for Prophet
        prophet_df = pd.DataFrame({
            'ds': df_input['timestamp'],
            'y': df_input['metric_0']
        })
        
        # Fit Prophet model
        model = Prophet(yearly_seasonality=False, daily_seasonality=False)
        model.fit(prophet_df)
        
        # Make forecast
        forecast = model.predict(prophet_df[['ds']])
        
        # Calculate residuals
        residuals = df_input['metric_0'].values - forecast['yhat'].values
        threshold = threshold_std * residuals.std()
        
        # Detect anomalies
        predictions = (np.abs(residuals) > threshold).astype(int)
        
        return predictions, model, forecast
    except Exception as e:
        logger.error(f"Prophet error: {e}")
        return None, None, None

# Apply Prophet
prophet_preds, prophet_model, prophet_forecast = detect_anomalies_prophet(df)

if prophet_preds is not None:
    logger.info(f"Prophet detected {prophet_preds.sum()} anomalies")
    precision = precision_score(df['label'], prophet_preds, zero_division=0)
    recall = recall_score(df['label'], prophet_preds, zero_division=0)
    f1 = f1_score(df['label'], prophet_preds, zero_division=0)
    print(f"Prophet Performance: Precision={precision:.3f}, Recall={recall:.3f}, F1={f1:.3f}")

### 4. Save Models

In [ ]:
# Save models locally
if arima_model is not None:
    with open(MODELS_DIR / 'arima_model.pkl', 'wb') as f:
        pickle.dump(arima_model, f)
    logger.info("Saved ARIMA model locally")

if prophet_model is not None:
    with open(MODELS_DIR / 'prophet_model.pkl', 'wb') as f:
        pickle.dump(prophet_model, f)
    logger.info("Saved Prophet model locally")

# Upload models to S3 for persistent storage
try:
    from common_functions import upload_model_to_s3, test_s3_connection
    
    if test_s3_connection():
        if arima_model is not None:
            upload_model_to_s3(
                str(MODELS_DIR / 'arima_model.pkl'),
                s3_key='models/anomaly-detection/arima_model.pkl'
            )
        if prophet_model is not None:
            upload_model_to_s3(
                str(MODELS_DIR / 'prophet_model.pkl'),
                s3_key='models/anomaly-detection/prophet_model.pkl'
            )
    else:
        logger.info("S3 not available - models saved locally only")
except ImportError:
    logger.info("S3 functions not available - models saved locally only")
except Exception as e:
    logger.warning(f"S3 upload failed (non-critical): {e}")

# Save predictions
results_df = pd.DataFrame({
    'actual': df['label'],
    'arima_pred': arima_preds if arima_preds is not None else 0,
    'prophet_pred': prophet_preds if prophet_preds is not None else 0
})
results_df.to_parquet(PROCESSED_DIR / 'timeseries_predictions.parquet')
logger.info("Saved predictions")

## Validation Section

In [ ]:
# Verify outputs
assert (MODELS_DIR / 'arima_model.pkl').exists(), "ARIMA model not saved"
assert (MODELS_DIR / 'prophet_model.pkl').exists(), "Prophet model not saved"
assert (PROCESSED_DIR / 'timeseries_predictions.parquet').exists(), "Predictions not saved"

logger.info("✅ All validations passed")
print(f"\nModels saved to: {MODELS_DIR}")
print(f"Predictions saved to: {PROCESSED_DIR}")

## Integration Section

This notebook integrates with:
- **Input**: Synthetic anomalies from `synthetic-anomaly-generation.ipynb`
- **Output**: Time series models for `ensemble-anomaly-methods.ipynb`
- **Coordination Engine**: Models can be deployed for real-time detection

## Next Steps

1. Review model performance metrics
2. Proceed to `lstm-based-prediction.ipynb` for deep learning approach
3. Compare with ensemble methods
4. Deploy best model to coordination engine

## References

- ADR-012: Notebook Architecture for End-to-End Workflows
- [ARIMA Documentation](https://www.statsmodels.org/stable/generated/statsmodels.tsa.arima.model.ARIMA.html)
- [Prophet Documentation](https://facebook.github.io/prophet/)